In [1]:
#!/usr/bin/env python3
#pip install  rioxarray==0.3.1
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray
import numpy as np
import geopandas as gpd
import cartopy.crs as ccrs
import rasterio
import os
import matplotlib.colors
scriptsdir = os.getcwd()
from scipy.interpolate import griddata
from functools import reduce
import xarray
import itertools
import argparse
import matplotlib.colors as mcolors



models = ["GBM"]
taxas = ["Amphibians"]
scenarios =["rcp60"]

In [19]:


model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
years = ['1845', '1990', '1995', '2009', '2010', '2020', '2026', '2032', '2048', '2050',
         '2052', '2056', '2080', '2100', '2150', '2200', '2250']
habitats=["forest","pasture","cropland","natural_land","nonforest"]


for habitat in habitats:
    for taxa in taxas:
        for model in models:
            df = pd.read_csv('/storage/homefs/ch21o450/scripts/BioScenComb/habitat_counts/habitat_'+ habitat + '_' + taxa + '.csv')
            species_list = df['Species'].tolist()  # Replace with the actual column name
            species_names = species_list[1000:2000]


In [7]:
species_names = ['Physalaemus_henselii']

In [23]:
def newvalue_fun(time, model, netcdf_path_format, is_historical=False, scenario=None):
    newvalue_dict = {model_name: {} for model_name in model_names}
    sum_bin_dict = {model_name: {} for model_name in model_names}

    for model_name in model_names:
        for species_name in species_names:
            try:
                if is_historical:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time), decode_times=False)
                else:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, model_name, scenario, species_name, time), decode_times=False)

                newvalue = ds["newvalue"]
                sum_bin = ds["sum_bin"]

                newvalue_dict[model_name][species_name] = newvalue
                sum_bin_dict[model_name][species_name] = sum_bin
            except Exception as e:
                print(f"Skipping species {species_name} for model {model_name} and time {time}. Error: {e}")

    projections_dict = {}

    for species_name in species_names:
        value_list = []
        for model_name in model_names:
            try:
                value_bin = newvalue_dict[model_name][species_name]
                value_list.append(value_bin)
            except KeyError:
                continue

        if not value_list:  # Check if the list is empty
            print(f"Skipping species {species_name} because no valid data found.")
            continue

        try:
            value_bin_concat = xr.concat(value_list, dim="model_name")
            mean_value_bin = value_bin_concat.mean(dim="model_name")
            projections_dict[species_name] = mean_value_bin
        except StopIteration:
            print(f"Skipping species {species_name} because the list is empty after concatenation.")
            continue

    value_bin_list = list(projections_dict.values())
    mean_value_bin = xr.concat(value_bin_list, dim="species").sum(dim="species")  # Ensemble mean over species
    mean_value_bin = mean_value_bin.where(mean_value_bin > 0, 0)
    return mean_value_bin

def calculate_mean(time, model, netcdf_path_format, is_historical=False, scenario=None):
    newvalue_dict = {model_name: {} for model_name in model_names}
    sum_bin_dict = {model_name: {} for model_name in model_names}
    lu_sum_bin_dict = {model_name: {} for model_name in model_names}

    for model_name in model_names:
        for species_name in species_names:
            try:
                if is_historical:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time), decode_times=False)
                else:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, model_name, scenario, species_name, time), decode_times=False)
                sum_bin = ds["sum_bin"]
                sum_bin_dict[model_name][species_name] = sum_bin
            except Exception as e:
                print(f"Skipping species {species_name} for model {model_name} and time {time}. Error: {e}")

    projections_dict = {}

    for species_name in species_names:
        sum_bin_list = []
        for model_name in model_names:
            try:
                sum_bin = sum_bin_dict[model_name][species_name]
                sum_bin_list.append(sum_bin)
            except KeyError:
                continue

        if not sum_bin_list:  # Check if the list is empty
            print(f"Skipping species {species_name} because no valid data found.")
            continue

        try:
            sum_bin_concat = xr.concat(sum_bin_list, dim="model_name")
            mean_sum_bin = sum_bin_concat.mean(dim="model_name")
            projections_dict[species_name] = mean_sum_bin
        except StopIteration:
            print(f"Skipping species {species_name} because the list is empty after concatenation.")
            continue

    mean_sum_bin_list = list(projections_dict.values())
    mean_sum_bin = xr.concat(mean_sum_bin_list, dim="species").sum(dim="species")  # Ensemble mean over species
    mean_sum_bin = mean_sum_bin.where(mean_sum_bin > 0, 0)

    return mean_sum_bin


In [24]:

historical_time = 1146
future_times = [35, 65]
# scenarios = ["rcp26"]

netcdf_path_format_future = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/{}/{}/{}_[{}].nc"
netcdf_path_format_hist = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/EWEMBI/{}_[{}].nc"

mean_value_bin_hist = newvalue_fun(historical_time, model, netcdf_path_format_hist, is_historical=True)
mean_sum_bin_hist = calculate_mean(historical_time, model, netcdf_path_format_hist, is_historical=True)

mean_sum_bin_hist = mean_sum_bin_hist.isel(time=0)

year_indices = {1146: '1995', 35: '2050', 65: '2080', 85: '2100'}

for future_time in future_times:
    for scenario in scenarios:


        mean_value_bin_future = newvalue_fun(future_time, model, netcdf_path_format_future, is_historical=False, scenario=scenario)
        mean_sum_bin_future = calculate_mean(future_time, model, netcdf_path_format_future, is_historical=False, scenario=scenario)
        mean_sum_bin_future = mean_sum_bin_future.isel(time=0)


Skipping species Physalaemus_henselii for model GFDL-ESM2M and time 1146. Error: [Errno 2] No such file or directory: b'/storage/scratch/users/ch21o450/data/LandClim_Output/GBM/Amphibians/EWEMBI/Physalaemus_henselii_[1146].nc'
Skipping species Physalaemus_henselii for model IPSL-CM5A-LR and time 1146. Error: [Errno 2] No such file or directory: b'/storage/scratch/users/ch21o450/data/LandClim_Output/GBM/Amphibians/EWEMBI/Physalaemus_henselii_[1146].nc'
Skipping species Physalaemus_henselii for model HadGEM2-ES and time 1146. Error: [Errno 2] No such file or directory: b'/storage/scratch/users/ch21o450/data/LandClim_Output/GBM/Amphibians/EWEMBI/Physalaemus_henselii_[1146].nc'
Skipping species Physalaemus_henselii for model MIROC5 and time 1146. Error: [Errno 2] No such file or directory: b'/storage/scratch/users/ch21o450/data/LandClim_Output/GBM/Amphibians/EWEMBI/Physalaemus_henselii_[1146].nc'
Skipping species Physalaemus_henselii because no valid data found.
Skipping species Physalaemu